# NYC Taxi & Limo Trips

This notebook demonstrates the use of Polymer Widgets to visualize a sampling of New York City taxi trips in a dashboard layout.

To execute this notebook, you need to have the declarative widget Jupyter extension and its prerequisites installed. The README in the dashboard project contains instructions for running a Docker container with all the necessary bits.

N.B., After running all of the cells, click View → View Dashboard to interact with this notebook in a dashboard layout.

<div class="alert alert-info" role="alert" style="margin-top: 10px">
<strong>Note:</strong> Requires Declarative Widgets v0.6.0+.
</div>

## Setup

First we need to download and link to the third-party JS components for the widgets. These may take a bit to download on first run of this notebook.

In [ ]:
%%html
<link rel="import" href="urth_components/paper-dropdown-menu/paper-dropdown-menu.html" is='urth-core-import' package='PolymerElements/paper-dropdown-menu'>
<link rel="import" href="urth_components/paper-menu/paper-menu.html" is='urth-core-import' package='PolymerElements/paper-menu'>
<link rel="import" href="urth_components/paper-item/paper-item.html" is='urth-core-import' package='PolymerElements/paper-item'>
<link rel="import" href="urth_components/paper-slider/paper-slider.html" is='urth-core-import' package='PolymerElements/paper-slider'>
<link rel="import" href="urth_components/google-map/google-map.html" is='urth-core-import' package='GoogleWebComponents/google-map'>
<link rel="import" href="urth_components/google-map/google-map-marker.html" is='urth-core-import' package='GoogleWebComponents/google-map'>
<link rel="import" href="urth_components/urth-viz-table/urth-viz-table.html" is='urth-core-import'>
<link rel="import" href="urth_components/urth-viz-chart/urth-viz-chart.html" is='urth-core-import'>

<style>
    div.output_wrapper {
        z-index: auto; /* fixes menus showing under code cells */
    }
</style>

In [ ]:
import declarativewidgets as widgets

widgets.init()

N.B. We're using absolute paths here to allow this notebook to work even when deployed as a dashboard where Thebe requests a kernel. When it does, it has the Jupyter notebook working directory as the `pwd` in which case a relative path will not work here. 

This is one of the challenges to be addressed with dashboards when deployed outside the authoring environment: access to data. A simple answer: put the data in a real data store, not local disk.

In [ ]:
import pandas as pd
import os
LOCAL_DATA = '/home/jovyan/work/taxi_demo/datasets'

In [ ]:
def loadData(name):
    return pd.read_csv(os.path.join(LOCAL_DATA, name + '.csv'))

## Trips

To begin, select a sample of trips from the **Dataset** dropdown. This will load the data into a table. Click a row in the table to show the pickup and dropup location for the trip on a map.

In [ ]:
%%html
<template is="urth-core-bind">
    <urth-core-function id="loadDataFunc" 
        ref="loadData" 
        arg-name="{{dataname}}" 
        result="{{thedata}}" 
        limit="10"
        auto></urth-core-function>

    <paper-dropdown-menu id="dataset-select" label="Dataset" selected-item-label="{{dataname}}" noink>
        <paper-menu class="dropdown-content" selected="0">
            <paper-item>trips_1</paper-item>
            <paper-item>trips_2</paper-item>
            <paper-item>trips_3</paper-item>
        </paper-menu>
    </paper-dropdown-menu>
</template>

In [ ]:
%%html
<div style="overflow-x:scroll;">
    <template is='urth-core-bind'>
        <urth-viz-table datarows='[[thedata.data]]' columns='[[thedata.columns]]' selection='{{sel}}'></urth-viz-table>
    
        <template is="dom-if" if="[[thedata]]">
            <div class="table-desc">
                <h4>Data for <span>[[dataname]]</span></h4>
                <p>Click an entry in the table to map the pickup and dropoff locations.</p>
            </div>
            <style>
                .table-desc {
                    text-align: center;
                }
                .table-desc > * {
                    display: inline-block;
                    margin-left: 0.5em;
                }
                .table-desc > p {
                    margin-bottom: 1em; 
                    font-style: oblique;
                    font-size: smaller;
                }
            </style>
        </template>
    </template>
</div>

#### Use of "dom-if"

The `google-map` Polymer widget below doesn't work well if there is no selection (`sel`). And later when there is a selection, it doesn't update correctly.

To work around this issue, we use Polymer's [conditional template](https://www.polymer-project.org/1.0/docs/devguide/templates.html#dom-if), `dom-if`. For our use, this means that the `google-map` widget will not be created when `sel` is undefined. When the user makes a selection in the table above, the condition is met and the map is correctly drawn.

In [ ]:
%%html
<template is="urth-core-bind">
  <template is="dom-if" if="[[sel]]">
    <google-map id="blah" latitude="[[sel.11]]" longitude="[[sel.10]]" zoom="12" style="height: 400px;">
        <google-map-marker latitude="[[sel.11]]" longitude="[[sel.10]]" title="Pickup"></google-map-marker>
        <google-map-marker latitude="[[sel.13]]" longitude="[[sel.12]]" title="Dropoff"></google-map-marker>
    </google-map>
  </template>
</template>

## Fares

We can also investigate the fares collected by the taxis and limos.

In [ ]:
fares = pd.read_csv(os.path.join(LOCAL_DATA, 'fares.csv'))

Here's a small sample of the loaded fares data. We show this data by linking a pandas DataFrame to another clickable table.

In [ ]:
%%html
<div style="overflow-x:scroll;">
    <template is="urth-core-bind" channel="faresChan">
        <urth-core-dataframe ref="fares" value="{{faredata}}" limit="10" auto></urth-core-dataframe>
        <urth-viz-table datarows='[[faredata.data]]' columns='[[faredata.columns]]'></urth-viz-table>
    </template>
</div>

Now let's graph the top fare earners. Pick a fare type to both plot and show in another table.

In [ ]:
def loadFares(filter):
    if (filter == "Credit"):
        _fares = fares[fares[' payment_type']=='CRD']
    elif (filter == "Cash"):
        _fares = fares[fares[' payment_type']=='CSH']
    else:
        _fares = fares

    topEarners = pd.DataFrame(_fares.groupby('medallion').sum().sort(' total_amount', inplace=False, ascending=False)[' total_amount']) 
    topEarners['medallion'] = topEarners.index
    topEarners = topEarners[[' total_amount']]
    return topEarners.reset_index()

In [ ]:
%%html
<template is="urth-core-bind" channel="faresChan">
    <urth-core-function id="loadFaresFunc" 
        ref="loadFares" 
        arg-filter="{{faredataselect}}" 
        result="{{topEarners}}" 
        limit="10"
        auto></urth-core-function>

    <paper-dropdown-menu id="fare-select" label="Fare Type" selected-item-label="{{faredataselect}}" noink>
        <paper-menu class="dropdown-content" selected="0">
            <paper-item>All</paper-item>
            <paper-item>Credit</paper-item>
            <paper-item>Cash</paper-item>
        </paper-menu>
    </paper-dropdown-menu>
</template>

In [ ]:
%%html
<div style="overflow-x:scroll;">
    <template is="urth-core-bind" channel="faresChan">
        <urth-viz-table datarows='[[topEarners.data]]' columns='[[topEarners.columns]]'></urth-viz-table>
    </template>
</div>

In [ ]:
%%html
<template is="urth-core-bind" channel="faresChan">
    <urth-viz-chart type='bar' datarows='[[topEarners.data]]' columns='[[topEarners.columns]]' rotatelabels='30'></urth-viz-chart>
</template>